In [40]:
# General stuff
import os
import re
import time
import warnings
import math
import sys
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from subprocess import Popen, PIPE
from os.path import join as opj
from IPython.core.debugger import Tracer
from nilearn.decomposition import CanICA,DictLearning
from sklearn.decomposition import DictionaryLearning

# sklearn stuff
from sklearn.decomposition import TruncatedSVD,FastICA,SparsePCA,MiniBatchSparsePCA
from sklearn.utils.extmath import randomized_svd
from sklearn.linear_model import Ridge
from sklearn.decomposition import dict_learning_online
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import scale
import sklearn.metrics.cluster as metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster.hierarchical import _hc_cut # Internal function to cut ward tree, helps speed up things a lot
from sklearn.utils import resample
from sklearn.model_selection import KFold

# personal functions
from importlib import reload
import fsutils as fs
reload(fs)

if os.path.exists('/data1/vbeliveau/'):
    # On NRU server
    cluster_code='/data1/vbeliveau/5HT_clustering'
    cluster_data='/data1/vbeliveau/5HT_clustering_data'
    subjects_dir='/usr/local/nru/freesurfer/subjects'
elif os.path.exists('C:/Users/vbeliveau/'):
    # On laptop
    cluster_code='C:/Users/vbeliveau/Documents/GitHub/5HT_clustering'
    cluster_data='C:/Users/vbeliveau/Documents/5HT_clustering_data'
    subjects_dir='C:/Users/vbeliveau/Downloads/'
else:
    raise ValueError('Unknown location')
    
# Load local version of nilearn
if os.path.exists('/data1/vbeliveau/'):
    sys.path.append('/data1/vbeliveau/nilearn')
    import mynilearn.decomposition as dcm 
    reload(dcm)
elif os.path.exists('C:/Users/vbeliveau/'):
    sys.path.append('C:/Users/vbeliveau/Downloads/nilearn-master')
    import mynilearn.decomposition as dcm
    reload(dcm)
else:
    raise ValueError('Unknown location')
    
# Analysis directories
surf_data=opj(cluster_data,'surf_data')
fs.assert_dir(surf_data)
BPnd_data=opj(cluster_data,'BPnd')
fs.assert_dir(BPnd_data)
MFA_data=opj(cluster_data,'MFA')
MFA_preproc=opj(MFA_data,'preproc')
fs.assert_dir(MFA_data)
fs.assert_dir(MFA_preproc)

info_path=opj(cluster_code,'info_alltracers_base_healthy_hrrt.mat')
hemi_type=['lh','rh']

info=sio.loadmat(info_path)
tracers=['cumi','dasb','sb','az','C36']
petID=np.array([item for subl in info['petID'] for subsubl in subl for item in subsubl])
mrID=np.array([item for subl in info['mrID'] for subsubl in subl for item in subsubl])

In [ ]:
# Compute gradient structure for surfaces

targ_list=['fsaverage5']
surf_type='inflated'
hemi=['lh','rh']

for targ in targ_list:
    for hemi in hemi_type:
        fname=opj(subjects_dir,targ,'surf',hemi+'.'+surf_type)
        fmask=opj(surf_data,'mask.'+targ+'.'+hemi)
        fout=opj(surf_data,'gradstruct.'+targ+'.'+surf_type+'.'+hemi)
        proj,neigh,cortex,border=fs.surf_gradient_struct(fname,fmask,
                            verbose=True,validate_rotation=True,save_out=fout)

In [ ]:
# Compute gradient for BPnd

tracers=['cumi','dasb','sb','az','C36']
targ='fsaverage5'
hemi_type=['lh','rh']
sm='10'
surf_type='sphere'
fgrad=opj(surf_data,'gradstruct.'+targ+'.'+surf_type+'.'+hemi)

# Process BPnd
grad_dest=opj(BPnd_data,'gradient')
fs.assert_dir(grad_dest)
for tracer in tracers:
    tracer_dest=opj(grad_dest,tracer)
    fs.assert_dir(tracer_dest)    
    for hemi in hemi_type:
        # Compute area ratio
        for subj in subjlist:
            fname=opt(surf_data,'area',subj+'.ico'+ico+'.'+hemi+'.nii.gz')
            img=fs.load_surf_data()
        corrected=
        
        fi='mean.mrtm2.nopvc.' + targ+'.'+ hemi + '.sm' + sm + '.nii.gz'
        fo='mean.mrtm2.nopvc.' + targ+'.'+surf_type+'.'+ hemi + '.sm' + sm + '.nii.gz'
        fname=opj(BPnd_data,'maps',tracer,fi)        
        data=fs.load_surf_data(fname)
        save_out=opj(tracer_dest,'grad.'+fo)
        area_ratio=fs.surf_area_ratio()
        grad=fs.surf_gradient(data,fgrad,save_out=save_out,verbose=True)

In [12]:
a=re.search('^sb','dasb')
a

In [42]:
tracer='cumi'
ind=np.array([re.search('^'+tracer,x) is not None for x in petID])
subjlist=[x+'_GD' for x in mrID[ind]]

In [43]:
subjlist

['f6598_GD',
 'f5890_GD',
 'f6184_GD',
 'f6200_GD',
 'f6456_GD',
 'f6836_GD',
 'f6808_GD',
 'f6842_GD']